# Exercise 04 - Building a Database via Apache Arrow

Remember our graph from _Exercise 02_?

```
(:User)-[:HAS_IP]->(:IP)
```

Let's take the same dataset, but this time we're going to build a **database** using the `neo4j_arrow` client!

Recall that before we did:

```python
df = pd.read_parquet("https://storage.googleapis.com/neo4j-se-public/training/user.parquet")
users = pa.Table.from_pandas(df)

df = pd.read_parquet("https://storage.googleapis.com/neo4j-se-public/training/ip.parquet")
ips = pa.Table.from_pandas(df)

df = pd.read_parquet("https://storage.googleapis.com/neo4j-se-public/training/has_ip.parquet")
has_ip = pa.Table.from_pandas(df)
```

> ...or we used https urls.

Now you should use what you learned in _Exercise 03_ to do it the PyArrow way.

In [ ]:
%%capture
%pip install https://github.com/neo4j-field/neo4j_arrow/releases/download/0.1.1/neo4j_arrow-0.1.1.tar.gz
%pip install https://github.com/neo4j-field/checker/releases/download/0.4.1/checker-0.4.1.tar.gz

In [ ]:
import pandas as pd

import pyarrow as pa
import pyarrow.parquet
import answers.checker as c

from neo4j_arrow import Neo4jArrowClient

In [ ]:
# Update this if you're not running locally with the provided Docker instances.
USE_TLS = False
NEO4J_HOST = "neo4j.arrow"
NEO4J_URI = f"neo4j{'+s' * int(USE_TLS)}://{NEO4J_HOST}:7687"
NEO4J_AUTH = ("neo4j", "password")

In [ ]:
client = Neo4jArrowClient(NEO4J_HOST,    # host or ip
                          "Exercise-04", # graph projection name (still required for now)
                          port=8491,
                          tls=USE_TLS,
                          user=NEO4J_AUTH[0],
                          password=NEO4J_AUTH[1],
)

---
<br><br>

### Task 1: Prepare our Data as PyArrow Tables 🧮

Use what you learned in _Exercise 03_ to load the `User`, `IP`, and `HAS_IP` datasets. Call them `users`, `ips`, and `has_ip` respectively.

In [ ]:
df = None # Finish me
users = None # Finish me

df = None # Finish me
ips = None # Finish me

df = None # Finish me
has_ip = None # Finish me

In [ ]:
# Don't change this cell.
c.check_result("Ex 04", "Task 1", users=users, ips=ips, has_ip=has_ip)

---
<br><br>

## Starting our Import

This is the primary difference from performing a GDS Graph projection. All we need to do is change some of the input parameters for the `start` message sent to the server.

> Note that there are both _different_ and _more_ parameters for database creation! It's common to have to tune these.

In [ ]:
config = {
    "name": "Exercise-04",
    "id_type": "INTEGER",        # can be: INTEGER or STRING
    "id_property": "nodeId",
    "record_format": "standard", # can be: standard, aligned, or high_limit
    "force": True                # overwrite existing db?
}

client.start("CREATE_DATABASE", config=config)

---
<br><br>

### Task 2: Load our Nodes

Basically all the steps from here on are the same as when building a GDS Projection. This means you already know how to do this, right? 😜

For the users, write them and capture the results as `users_results`. For IPs, use `ips_results`.

In [ ]:
users_results = None # Finish me
ips_results = None # Finish me

In [ ]:
# Don't change this cell.
c.check_result("Ex 04", "Task 2", 
               users_results=users_results, ips_results=ips_results)

---
<br><br>

### Task 3: Signal that We're Done with Nodes

Easy, right? Now signal we're done with nodes and capture the result as `result`.

In [ ]:
result = None # Finish me

In [ ]:
# Don't change this cell.
c.check_result("Ex 04", "Task 3", result = result)

---
<br><br>

### Task 4: Send our Relationships

By now, you should know the routine, right? Write the edges and store the response in `result`, please!

In [ ]:
result = None # Finish me

In [ ]:
# Don't change this cell.
c.check_result("Ex 04", "Task 4", result = result)

---
<br><br>

### Task 5: Signal our Loading is Done

This should be easy by now. You've learned so much. Look at you go.

Signal we're done with edges and assign the result message to `result`.

In [ ]:
result = None # Finish me

In [ ]:
# Don't change this cell.
c.check_result("Ex 04", "Task 5", result = result)

---
<br><br>

## Validating our Work

Now we need to check that we have an actual **database**. The easiest way to do this is to just fire off a Cypher query.

But, is the database online yet? The import process is **asynchronous** at this point. Our client is told only that the GDS service has accepted our nodes and relationships.

In this case, our dataset is small (~600k nodes, 1.5M edges) and with few properties. 

In [ ]:
from graphdatascience import GraphDataScience
from time import sleep

gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

# Database import is asynchronous. We need to await it's being brought online.
# NOTE: it appears names may be slightly changed (uppercase to lowercase)...might be a bug.
times = 0
while times < 20:
    dbs = gds.run_cypher(
        "SHOW DATABASES YIELD name, currentStatus WHERE toLower(name) = $name;", 
        params={"name": "exercise-04"})
    if dbs.empty:
        times += 1
        sleep(1)
    else:
        break

if dbs.empty:
    raise Exception("database didn't come online! check logs!")
print("Found our database online!")

In [ ]:
# Check that we have data!
gds.set_database("Exercise-04")

df = gds.run_cypher("""
    MATCH (u:User)-[r:HAS_IP]->(i:IP)
    WHERE id(i) = 33968
    RETURN id(u) AS user, u.fraudMoneyTransfer, id(r) AS has_ip, id(i) AS ip
    LIMIT 10
""")
df

In [ ]:
# Drop our database. 👋
gds.run_cypher("DROP DATABASE `exercise-04`;")

## Conclusion

You should see the following logging on the Neo4j server indicating the various import steps the batch importer performed during database creation:

```
2022-09-21 19:51:51.647+0000 INFO  [system/00000000] Database import started
2022-09-21 19:51:51.671+0000 INFO  [system/00000000] Put command: PutCommand{name=Exercise-04, entityType=node}
2022-09-21 19:51:51.682+0000 INFO  [system/00000000] Put stream started
2022-09-21 19:51:51.683+0000 INFO  [system/00000000] Put command: PutCommand{name=Exercise-04, entityType=node}
2022-09-21 19:51:51.832+0000 INFO  [system/00000000] Received action NODE_LOAD_DONE with configuration NodeLoadDoneAction{name=Exercise-04}
2022-09-21 19:51:51.842+0000 INFO  [system/00000000] Put stream started
2022-09-21 19:51:51.843+0000 INFO  [system/00000000] Put command: PutCommand{name=Exercise-04, entityType=relationship}
2022-09-21 19:51:52.308+0000 INFO  [system/00000000] Received action RELATIONSHIP_LOAD_DONE with configuration RelationshipLoadDoneAction{name=Exercise-04}
2022-09-21 19:51:53.036+0000 INFO  [system/00000000] Nodes :: Started
2022-09-21 19:51:54.788+0000 INFO  [system/00000000] Nodes :: Finished
2022-09-21 19:51:54.797+0000 INFO  [system/00000000] Prepare node index :: Started
2022-09-21 19:51:55.195+0000 INFO  [system/00000000] Prepare node index :: Finished
2022-09-21 19:51:55.253+0000 INFO  [system/00000000] Relationships :: Started
2022-09-21 19:51:57.888+0000 INFO  [system/00000000] Relationships :: Finished
2022-09-21 19:51:57.914+0000 INFO  [system/00000000] Node Degrees :: Started
2022-09-21 19:51:58.127+0000 INFO  [system/00000000] Node Degrees :: Finished
2022-09-21 19:51:58.247+0000 INFO  [system/00000000] Relationship --> Relationship :: Started
2022-09-21 19:51:58.622+0000 INFO  [system/00000000] Relationship --> Relationship :: Finished
2022-09-21 19:51:58.634+0000 INFO  [system/00000000] RelationshipGroup :: Started
2022-09-21 19:51:58.762+0000 INFO  [system/00000000] RelationshipGroup :: Finished
2022-09-21 19:51:58.781+0000 INFO  [system/00000000] Node --> Relationship :: Started
2022-09-21 19:51:59.088+0000 INFO  [system/00000000] Node --> Relationship :: Finished
2022-09-21 19:51:59.134+0000 INFO  [system/00000000] Relationship <-- Relationship :: Started
2022-09-21 19:51:59.510+0000 INFO  [system/00000000] Relationship <-- Relationship :: Finished
2022-09-21 19:51:59.546+0000 INFO  [system/00000000] Count groups :: Started
2022-09-21 19:51:59.556+0000 INFO  [system/00000000] Count groups :: Finished
2022-09-21 19:51:59.608+0000 INFO  [system/00000000] Gather :: Started
2022-09-21 19:51:59.639+0000 INFO  [system/00000000] Gather :: Finished
2022-09-21 19:51:59.643+0000 INFO  [system/00000000] Write :: Started
2022-09-21 19:51:59.653+0000 INFO  [system/00000000] Write :: Finished
2022-09-21 19:51:59.681+0000 INFO  [system/00000000] Node --> Group :: Started
2022-09-21 19:51:59.691+0000 INFO  [system/00000000] Node --> Group :: Finished
2022-09-21 19:51:59.729+0000 INFO  [system/00000000] Node counts and label index build :: Started
2022-09-21 19:52:00.074+0000 INFO  [system/00000000] Node counts and label index build :: Finished
2022-09-21 19:52:00.109+0000 INFO  [system/00000000] Relationship counts and relationship type index build :: Started
2022-09-21 19:52:00.612+0000 INFO  [system/00000000] Relationship counts and relationship type index build :: Finished
2022-09-21 19:52:00.732+0000 INFO  [system/00000000] Database import finished after 9083 ms
2022-09-21 19:52:01.837+0000 INFO  [system/00000000] Database created and started after 1104 ms
```